# Load to qdrant

In [1]:
from qdrant_client import QdrantClient, models
import pandas as pd
import numpy as np
import os

In [2]:
client = QdrantClient(url="http://localhost:6333")
client

In [3]:
basePath = os.path.abspath('')
basePath

'C:\\Users\\gorku\\Documents\\bmstu\\RAG24\\late-chunking-ru'

In [6]:
client.create_collection(
    collection_name="{chunks_embedded_trad}",
    vectors_config=models.VectorParams(size=1024, distance=models.Distance.COSINE, on_disk=True),
    on_disk_payload=True,
    optimizers_config=models.OptimizersConfigDiff(
        indexing_threshold=0,
    ),
)

True

In [28]:
'''
client.create_collection(
    collection_name="{chunks_embedded_new}",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE, on_disk=True),
    on_disk_payload=True,
    optimizers_config=models.OptimizersConfigDiff(
        indexing_threshold=0,
    ),
)
'''

True

In [5]:
client.delete_collection(collection_name="{chunks_embedded_trad}")

True

In [7]:
chunks_embedded_n = pd.read_pickle(basePath + "\\ai-forever-ria-news-retrieval\\chunks_embedded_0.pkl")

In [8]:
chunks_embedded_n.head()

,chunk,trad_chunk_embedding,new_chunk_embedding,doc_id
0,"премьер-министр украины, кандидат в президенты...","[0.005897517, -0.027958157, -0.02522486, 0.033...","[-0.20117188, -0.83984375, -0.80859375, 1.0156...",0
1,17 января в украине состоялся первый тур выб...,"[0.06695399, -0.112217665, -0.07667681, 0.0784...","[0.20800781, -0.90234375, -1.1796875, 1.140625...",0
2,второй тур выборов президента украины состои...,"[0.034322843, -0.07454268, -0.03999842, 0.0665...","[0.41210938, -0.9140625, -0.6796875, 1.21875, ...",0
3,парламент украины по инициативе партии регион...,"[-0.011248906, -0.03999717, -0.08579018, 0.054...","[-0.013244629, -0.84765625, -1.1015625, 1.0703...",0
4,министра.,"[0.03110334, -0.14622205, 0.027806075, 0.01278...","[-0.12207031, -1.2265625, -1.3203125, 0.714843...",0


In [9]:
client.get_collection(collection_name="{chunks_embedded_trad}")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=0, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=True, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=0, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantiza

In [ ]:
operation_info = client.upsert(
    collection_name="{chunks_embedded_trad}",
    wait=True,
    points=[
        models.PointStruct(
            id=id, vector=row['trad_chunk_embedding'], payload={"chunk": row['chunk'], "doc_id": row['doc_id']}
        ) for id, row in chunks_embedded_n.iterrows()
    ],
)

print(operation_info)

In [ ]:
client.close()

In [ ]:
client.update_collection(
    collection_name="{chunks_embedded_trad}",
    optimizer_config=models.OptimizersConfigDiff(indexing_threshold=20000),
)

In [ ]:
client.update_collection(
    collection_name="{chunks_embedded_new}",
    optimizer_config=models.OptimizersConfigDiff(indexing_threshold=20000),
)

In [ ]:
'''
client.create_collection(
    collection_name="{chunks_embedded}",
    sparse_vectors={
        "text": models.SparseVectorIndexParams(
            index=models.SparseVectorIndexType(
                on_disk=False,
            ),
        ),
    },
)
'''

In [ ]:
queries = pd.read_json(basePath + "\\ai-forever-ria-news-retrieval\\queries.jsonl", lines=True)
corpus = pd.read_json(basePath + "\\ai-forever-ria-news-retrieval\\corpus.jsonl", lines=True)
test = pd.read_json(basePath + "\\ai-forever-ria-news-retrieval\\test.jsonl", lines=True)
del corpus['title']
del test['score']

In [ ]:
chunked_corpus  = np.array_split(corpus, 22) # Пусть у нас 5179 компьютер, сольём первые два из них для примера

In [ ]:
cos_sim = lambda x, y: np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))
